https://gist.github.com/dirko/1d596ca757a541da96ac3caa6f291229

http://dirko.github.io/Bidirectional-LSTMs-with-Keras/

In [1]:
# author: Keras==1.0.6
# mine: Keras==1.2.1
import numpy as np
import re

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import TimeDistributedDense, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.layers import Merge
from keras.backend import tf

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split

from lambdawithmask import Lambda as MaskLambda

Using TensorFlow backend.
/Users/jzhu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def encode(x, n):
    result = np.zeros(n)
    result[x] = 1
    return result

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        
def reverse_func(x, mask=None):
    return tf.reverse(x, [False, True, False])

def score1(yh, pr):
    coords = [np.where(yhh > 0)[0][0] for yhh in yh]
    yh = [yhh[co:] for yhh, co in zip(yh, coords)]
    ypr = [prr[co:] for prr, co in zip(pr, coords)]
    fyh = [c for row in yh for c in row]
    fpr = [c for row in ypr for c in row]
    return fyh, fpr

In [3]:
raw = open('train.csv', 'r').readlines()
all_x = []
point = []
for line in raw:
    stripped_line = line.strip().split(',')
    point.append(stripped_line)
    if line == '""\r\n':
#         print "newline"
        all_x.append(point[:-1])
        point = []
all_x = all_x[:-1]
lengths = [len(x) for x in all_x]
# short_x = [x for x in all_x if len(x) < 64]

# split long sections into chucks (a mimic of sentences)
short_x = []
for l in all_x:
    short_x.extend(chunks(l, 64))

In [4]:
len(short_x)

3428

In [5]:
X = [[c[0] for c in x] for x in short_x]
y = [[c[1] for c in y] for y in short_x]

In [6]:
all_text = [c for x in X for c in x]
words = list(set(all_text))
word2ind = {word: index for index, word in enumerate(words)}
ind2word = {index: word for index, word in enumerate(words)}
labels = list(set([c for x in y for c in x]))
# label2ind = {label: (index + 1) for index, label in enumerate(labels)}
# ind2label = {(index + 1): label for index, label in enumerate(labels)}
label2ind = {label: (index) for index, label in enumerate(labels)}
ind2label = {(index): label for index, label in enumerate(labels)}
print 'Input sequence length range: ', max(lengths), min(lengths)

Input sequence length range:  4741 58


In [7]:
ind2label

{0: 'Severity',
 1: 'Negation',
 2: 'O',
 3: 'DrugClass',
 4: 'Animal',
 5: 'Factor',
 6: 'AdverseReaction'}

In [8]:
maxlen = max([len(x) for x in X])
print 'Maximum sequence length:', maxlen

Maximum sequence length: 64


In [9]:
X_enc = [[word2ind[c] for c in x] for x in X]
X_enc_reverse = [[c for c in reversed(x)] for x in X_enc]
max_label = max(label2ind.values()) + 1
# max_label = max(label2ind.values())
y_enc = [[0] * (maxlen - len(ey)) + [label2ind[c] for c in ey] for ey in y]
y_enc = [[encode(c, max_label) for c in ey] for ey in y_enc]

In [10]:
max_label

7

In [11]:
y_enc[0][0]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.])

In [12]:
X_enc_f = pad_sequences(X_enc, maxlen=maxlen)
X_enc_b = pad_sequences(X_enc_reverse, maxlen=maxlen)
y_enc = pad_sequences(y_enc, maxlen=maxlen)

In [13]:
(X_train_f, X_test_f, X_train_b,
 X_test_b, y_train, y_test) = train_test_split(X_enc_f, X_enc_b, y_enc,
                                               test_size=11*32, train_size=45*32, random_state=42)
print 'Training and testing tensor shapes:'
print X_train_f.shape, X_test_f.shape, X_train_b.shape, X_test_b.shape, y_train.shape, y_test.shape

Training and testing tensor shapes:
(1440, 64) (352, 64) (1440, 64) (352, 64) (1440, 64, 7) (352, 64, 7)


In [14]:
max_features = len(word2ind)
embedding_size = 128
hidden_size = 32
out_size = len(label2ind) #+ 1

In [15]:
model_forward = Sequential()
model_forward.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
model_forward.add(LSTM(hidden_size, return_sequences=True))  

In [16]:
model_backward = Sequential()
model_backward.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
model_backward.add(LSTM(hidden_size, return_sequences=True))
model_backward.add(MaskLambda(function=reverse_func, mask_function=reverse_func))

In [78]:
model = Sequential()

model.add(Merge([model_forward, model_backward], mode='concat'))
model.add(TimeDistributedDense(out_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [79]:
batch_size = 64 #32
model.fit([X_train_f, X_train_b], y_train, batch_size=batch_size, nb_epoch=30, # started from 40
          validation_data=([X_test_f, X_test_b], y_test))
score = model.evaluate([X_test_f, X_test_b], y_test, batch_size=batch_size)
print('Raw test score:', score)

Train on 1440 samples, validate on 352 samples
Epoch 1/30
1440/1440 [==============================] - 8s - loss: 0.6634 - val_loss: 0.3467
Epoch 2/30
1440/1440 [==============================] - 5s - loss: 0.2302 - val_loss: 0.2688
Epoch 3/30
1440/1440 [==============================] - 5s - loss: 0.1603 - val_loss: 0.2350
Epoch 4/30
1440/1440 [==============================] - 5s - loss: 0.1262 - val_loss: 0.2210
Epoch 5/30
1440/1440 [==============================] - 6s - loss: 0.1081 - val_loss: 0.2136
Epoch 6/30
1440/1440 [==============================] - 5s - loss: 0.0960 - val_loss: 0.2054
Epoch 7/30
1440/1440 [==============================] - 6s - loss: 0.0861 - val_loss: 0.2006
Epoch 8/30
1440/1440 [==============================] - 5s - loss: 0.0782 - val_loss: 0.2010
Epoch 9/30
1440/1440 [==============================] - 6s - loss: 0.0716 - val_loss: 0.1953
Epoch 10/30
1440/1440 [==============================] - 6s - loss: 0.0663 - val_loss: 0.1960
Epoch 11/30
1440/1440 

In [80]:
pr = model.predict_classes([X_train_f, X_train_b])
yh = y_train.argmax(2)
fyh, fpr = score1(yh, pr)
print 'Training accuracy:', accuracy_score(fyh, fpr)
print 'Training confusion matrix:'
print confusion_matrix(fyh, fpr)

1440/1440 [==============================] - 2s     
Training accuracy: 0.999458880559
Training confusion matrix:
[[  366     0     1     0     0     0     0]
 [    1    34     7     0     0     0     0]
 [    1     0 82145     0     0     1     4]
 [    0     0     8    91     0     0     0]
 [    0     0     2     0    12     0     0]
 [    1     0    13     0     0   252     0]
 [    4     0     5     0     0     0  5757]]


In [81]:
pr = model.predict_classes([X_test_f, X_test_b])
yh = y_test.argmax(2)
fyh, fpr = score1(yh, pr)
print 'Testing accuracy:', accuracy_score(fyh, fpr)
print 'Testing confusion matrix:'
print confusion_matrix(fyh, fpr)

352/352 [==============================] - 0s     
Testing accuracy: 0.972151311209
Testing confusion matrix:
[[   43     0    38     0     0     0     1]
 [    0     2    11     0     0     0     0]
 [   19     0 19835     2     0    14    70]
 [    1     0    22    11     1     0     0]
 [    0     0     0     0     3     0     0]
 [    0     0    29     0     0     9     0]
 [    3     0   389     0     0     0  1042]]


In [82]:
ind2label

{0: 'Severity',
 1: 'Negation',
 2: 'O',
 3: 'DrugClass',
 4: 'Animal',
 5: 'Factor',
 6: 'AdverseReaction'}

# Label the predictions

In [86]:
doc = "Classical HL post-auto-HSCT consolidation: neutropenia, peripheral sensory neuropathy, thrombocytopenia, anemia, upper respiratory tract infection, fatigue, peripheral motor neuropathy, nausea, cough, and diarrhea."
x_new = [[m.group(0) for m in re.finditer(r'\w+', doc)]] # tokenize words only

X_new = [[word2ind[c] for c in x] for x in x_new]
X_new_reverse = [[c for c in reversed(x)] for x in X_new]
X_new_f = pad_sequences(X_new, maxlen=maxlen)
X_new_b = pad_sequences(X_new_reverse, maxlen=maxlen)
print X_new_f.shape, X_new_b.shape

(1, 64) (1, 64)


In [87]:
pr = model.predict_classes([X_new_f, X_new_b])
pr

1/1 [==============================] - 0s


array([[2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 6, 2, 2, 2, 2, 6, 6, 6, 2, 6, 6, 6,
        0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2,
        2, 2, 2, 2, 6, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 6, 6, 2, 2]])

In [88]:
for w, p in zip(x_new[0], pr.tolist()[0][-len(X_new[0]):]):
    print w, ind2label[p]

Classical O
HL O
post O
auto O
HSCT O
consolidation O
neutropenia O
peripheral O
sensory AdverseReaction
neuropathy O
thrombocytopenia O
anemia AdverseReaction
upper O
respiratory O
tract O
infection O
fatigue O
peripheral O
motor O
neuropathy O
nausea AdverseReaction
cough AdverseReaction
and O
diarrhea O


### Issues:
* Cannot identify names with more than one word (how?)
* Two classes are missing in training data (why?)
* How to calculate and improve F1 score? Overfitting? How to tune parameters?
* Imbalanced dataset?

# Write submission file

https://bionlp.nlm.nih.gov/tac2017adversereactions/